In [5]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [6]:
lines= pd.read_table('allsimplewords.txt', names=['eng', 'tel'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [7]:
lines.shape

(2533, 2)

In [8]:
lines.eng=lines.eng.str.lower()#apply(lambda x: x.lower())
lines.tel=lines.tel.str.lower()

In [9]:
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', str(x)))
lines.tel=lines.tel.apply(lambda x: re.sub("'", '', str(x)))

In [10]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.tel=lines.tel.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [11]:
lines.eng

0               word
1              అంచనా
2             అంటాడు
3             అంటారు
4               అంటూ
5               అంటే
6             అంటేనే
7              అండగా
8              అండ్‌
9              అంతకు
10             అంతగా
11           అంతర్గత
12        అంతర్గతంగా
13              అంతా
14              అంతే
15           అందరికీ
16             అందరూ
17             అందాక
18            అందాలు
19              అంది
20      అకస్మాత్తుగా
21             అక్కడ
22          అక్కడికీ
23            అక్కడే
24       అజాగ్రత్తలు
25               అటూ
26          అడపాదడపా
27           అడిగాడు
28           అడిగారు
29             అడిగి
            ...     
2503       సర్దుకొని
2504          సర్ధడం
2505            సలహా
2506          సాకుతో
2507        సానుభూతి
2508        సాయంకాలం
2509            సార్
2510        సింగడికి
2511        సింగడిలో
2512          సింగడు
2513            సిఎం
2514        సిద్ధంగా
2515              సీ
2516      సున్నితంగా
2517          సూసైడ్
2518         సోమవారం
2519         

In [12]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.tel = lines.tel.apply(lambda x: re.sub("[0123456789]", "", x))

In [13]:
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.tel=lines.tel.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.tel=lines.tel.apply(lambda x: re.sub(" +", " ", x))

lines.tel = lines.tel.apply(lambda x : 'S'+ x + 'E')

In [14]:
lines.sample(10)

,eng,tel
859,తెలుసుకుంటుండాలి,SతెలియుE
522,క్షణం,Sక్షణంE
1327,మరచిపోకూడదు,SమరచిE
258,ఉత్తేజాన్ని,Sఉత్తేజంE
2293,ఫంక్షన్,Sఫంక్షన్E
26,అడపాదడపా,SఅడపాదడపాE
1866,అభ్యంతరం,Sఅభ్యంతరంE
525,ఖాళీగా,SఖాళీE
2517,సూసైడ్,Sసూసైడ్E
1997,కడుక్కొని,SకడుగుE


In [15]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng:
        if word not in all_eng_words:
            all_eng_words.add(word)
all_eng_words

{'d',
 'o',
 'r',
 'w',
 'ం',
 'ః',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఎ',
 'ఏ',
 'ఐ',
 'ఒ',
 'ఓ',
 'ఔ',
 'క',
 'ఖ',
 'గ',
 'ఘ',
 'చ',
 'ఛ',
 'జ',
 'ఞ',
 'ట',
 'ఠ',
 'డ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ',
 'ా',
 'ి',
 'ీ',
 'ు',
 'ూ',
 'ృ',
 'ె',
 'ే',
 'ై',
 'ొ',
 'ో',
 'ౌ',
 '్',
 'ౖ',
 '\u200c'}

In [16]:
all_Telugu_words=set()
for tel in lines.tel:
    for word in tel:
        if word not in all_Telugu_words:
            all_Telugu_words.add(word)
all_Telugu_words

{'E',
 'S',
 'g',
 'h',
 'r',
 't',
 'u',
 'ం',
 'ః',
 'అ',
 'ఆ',
 'ఇ',
 'ఈ',
 'ఉ',
 'ఊ',
 'ఎ',
 'ఏ',
 'ఐ',
 'ఒ',
 'ఓ',
 'ఔ',
 'క',
 'ఖ',
 'గ',
 'ఘ',
 'చ',
 'ఛ',
 'జ',
 'ఞ',
 'ట',
 'ఠ',
 'డ',
 'ణ',
 'త',
 'థ',
 'ద',
 'ధ',
 'న',
 'ప',
 'ఫ',
 'బ',
 'భ',
 'మ',
 'య',
 'ర',
 'ల',
 'ళ',
 'వ',
 'శ',
 'ష',
 'స',
 'హ',
 'ా',
 'ి',
 'ీ',
 'ు',
 'ూ',
 'ృ',
 'ె',
 'ే',
 'ై',
 'ొ',
 'ో',
 'ౌ',
 '్',
 'ౖ',
 '\u200c'}

In [17]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l))
max_length_src = np.max(lenght_list)
print(max_length_src)

18


In [18]:
lenght_list=[]
for l in lines.tel:
    lenght_list.append(len(l))
max_length_tar = np.max(lenght_list)

print(max_length_tar)

17


In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_Telugu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_Telugu_words)
print(num_encoder_tokens, num_decoder_tokens)
num_decoder_tokens += 1

64 67


In [20]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [21]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [22]:
lines = shuffle(lines)
print(lines.head(10))

                 eng         tel
648             చెడో      SచెడుE
1690          శారీరక   SశారీరముE
65        అనర్ధాన్ని   Sఅనర్థముE
576            గ్రామ    Sగ్రామంE
483         కొట్టాను    Sకొట్టుE
1746  సాగిపోతున్నాయి  SసాగిపోవుE
329             ఏందే      SఏంటిE
2210              నా        SనాE
1461           రెండూ     SరెండుE
343              ఏయ్       Sఏయ్E


In [23]:
X, y = lines.eng, lines.tel
X_train, X_testv, y_train, y_testv = train_test_split(X, y, test_size = 0.2)
X_test,X_val,y_test,y_val=train_test_split(X_testv,y_testv,test_size=0.5)
print(X_train.shape, X_test.shape,X_val.shape)

(2026,) (253,) (254,)


In [24]:
X_train.to_pickle('Weights_Tel/X_train.pkl')
X_test.to_pickle('Weights_Tel/X_test.pkl')

In [25]:
def generate_batch(X = X_train, y = y_train, batch_size = 10):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src+1),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar+1),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar+1, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text):
                    if t<len(target_text)-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [26]:
latent_dim = 50

In [27]:

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [28]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [42]:
train_samples = len(X_train)
val_samples = len(X_val)
batch_size = 1
epochs = 50

train_samples,val_samples

(2026, 254)

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = int(train_samples/batch_size),validation_steps = int(val_samples/batch_size),
                    epochs=epochs,
                    validation_data = generate_batch(X_val, y_val, batch_size = batch_size))


Epoch 1/50
2026/2026 [==============================] - 103s 51ms/step - loss: 0.0117 - acc: 0.9961 - val_loss: 0.0175 - val_acc: 0.9946
Epoch 2/50
2026/2026 [==============================] - 105s 52ms/step - loss: 0.0078 - acc: 0.9975 - val_loss: 0.0162 - val_acc: 0.9953
Epoch 3/50
2026/2026 [==============================] - 107s 53ms/step - loss: 0.0071 - acc: 0.9977 - val_loss: 0.0168 - val_acc: 0.9950
Epoch 4/50
2026/2026 [==============================] - 107s 53ms/step - loss: 0.0068 - acc: 0.9978 - val_loss: 0.0166 - val_acc: 0.9950
Epoch 5/50
2026/2026 [==============================] - 111s 55ms/step - loss: 0.0063 - acc: 0.9980 - val_loss: 0.0166 - val_acc: 0.9951
Epoch 6/50
2026/2026 [==============================] - 113s 56ms/step - loss: 0.0062 - acc: 0.9980 - val_loss: 0.0165 - val_acc: 0.9953
Epoch 7/50
1135/2026 [===============>..............] - ETA: 51s - loss: 0.0056 - acc: 0.9983

In [32]:
model.save_weights('nmt_weights.h5')

In [33]:
model.load_weights('nmt_weights.h5')

In [34]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [35]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    #print(target_token_index)
    target_seq[0, 0] = target_token_index['S']
   

    #print(target_seq)
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'E'  or len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        #print(sampled_token_index)
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [40]:
train_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [41]:
count=0
while(k<252):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    inp=X_test[k:k+1].values[0]
    act=y_test[k:k+1].values[0][1:-1]
    pred=decoded_sentence[:-1]
    print('Input English sentence:', inp)
    print('Actual Tel Translation:',act )
    print('Predicted Tel Translation:', pred)
    print(k)
    if(act==pred):
        count+=1
count

Input English sentence: కప్ప
Actual Tel Translation: కప్ప
Predicted Tel Translation: క్పూ
0
Input English sentence: మనసుతో
Actual Tel Translation: మనసు
Predicted Tel Translation: మనసు
1
Input English sentence: సీట్లు
Actual Tel Translation: సీటు
Predicted Tel Translation: సీటు
2
Input English sentence: కొట్టింది
Actual Tel Translation: కొట్టు
Predicted Tel Translation: కొట్టి
3
Input English sentence: కుదుపునకు
Actual Tel Translation: కుదుపు
Predicted Tel Translation: కుదుపు
4
Input English sentence: ప్లేటును
Actual Tel Translation: ప్లేటు
Predicted Tel Translation: ప్లేటు
5
Input English sentence: తగిలి
Actual Tel Translation: తగులు
Predicted Tel Translation: తగులు
6
Input English sentence: కాఫీ
Actual Tel Translation: కాఫి
Predicted Tel Translation: కావో
7
Input English sentence: ఐ
Actual Tel Translation: ఐ
Predicted Tel Translation: సక
8
Input English sentence: ముప్పిరిగొన్నాయి
Actual Tel Translation: ముప్పిరిగొను
Predicted Tel Translation: ముప్పద్తి
9
Input English sentence: వీరిలో

Input English sentence: ఏందే
Actual Tel Translation: ఏంటి
Predicted Tel Translation: ఏంది
88
Input English sentence: చూడక
Actual Tel Translation: చూడు
Predicted Tel Translation: చూడు
89
Input English sentence: సూచనలను
Actual Tel Translation: సూచన
Predicted Tel Translation: సూచన
90
Input English sentence: అందాలు
Actual Tel Translation: అందం
Predicted Tel Translation: అందా
91
Input English sentence: జగదీశ్
Actual Tel Translation: జగదీశ్
Predicted Tel Translation: జాటపసు
92
Input English sentence: వాళ్లకే
Actual Tel Translation: వాళ్లు
Predicted Tel Translation: వాళ్లు
93
Input English sentence: కారుకి
Actual Tel Translation: కారు
Predicted Tel Translation: కారు
94
Input English sentence: వచ్చాం
Actual Tel Translation: వచ్చు
Predicted Tel Translation: వచ్చి
95
Input English sentence: గొప్పవారో
Actual Tel Translation: గొప్ప
Predicted Tel Translation: గొప్పిడు
96
Input English sentence: అదే
Actual Tel Translation: అది
Predicted Tel Translation: అది
97
Input English sentence: కొత్త
Actual Te

Input English sentence: పెద్దగా
Actual Tel Translation: పెద్ద
Predicted Tel Translation: పెద్ద
175
Input English sentence: విభజించి
Actual Tel Translation: విభజన
Predicted Tel Translation: విజజంచు
176
Input English sentence: పాఠాలను
Actual Tel Translation: పాఠము
Predicted Tel Translation: పాఠము
177
Input English sentence: దిగితే
Actual Tel Translation: దిగు
Predicted Tel Translation: దిగుతు
178
Input English sentence: ప్రాంతం
Actual Tel Translation: ప్రాంతం
Predicted Tel Translation: ప్రాంతం
179
Input English sentence: ప్రేరేపిస్తాయ్‌
Actual Tel Translation: ప్రేరేపణ
Predicted Tel Translation: ప్రేరేపించు
180
Input English sentence: జవాబు
Actual Tel Translation: జవాబు
Predicted Tel Translation: జావసం
181
Input English sentence: ఆదిత్యను
Actual Tel Translation: ఆదిత్య
Predicted Tel Translation: ఆదిత్య
182
Input English sentence: అడపాదడపా
Actual Tel Translation: అడపాదడపా
Predicted Tel Translation: అడుపు
183
Input English sentence: తగదు
Actual Tel Translation: తగదు
Predicted Tel Translati

105